In [ ]:
# llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
# llm_name_short = "gpt-4o-mini_"
llm_dic = {
    # "openai_o3mini": "o3-mini-2025-01-31",
    # "gpt-4o-mini_": "gpt-4o-mini-2024-07-18",
    "llama8b_": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
    "mistral24b_": "mistral-small-2503",  # "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "llama70b_": "meta-llama/Llama-3.3-70B-Instruct-Turbo",
}
best_70b = "llama70b_social_n2_Tee_Sddd"
best_8b = "llama8b_social_n3_Teee_Srdd"
best_mistral = "mistral24b_social_n3_Teeo_Sddr"

the_bests = [best_70b, best_8b]  # best_70b
import glob

len(glob.glob("output/elbaff_experiment/final_states/*_2S*.json"))

0

In [ ]:
import json
import os, getpass
from langchain.chat_models import init_chat_model
from IPython.display import Image, display


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")
_set_env("TOGETHER_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"

In [ ]:
from glob import glob
import json
import time
from IPython.display import Image, display

from social_agents.agent_builder import (
    RankerAgentBuilder,
    TwoStepsCriteriaScorer,
    ValidatorAgentBuilder,
)

# from validator_agents.QuestionScorer import QuestionScorer
VALIDATION_4SCORES_VAL = "4SCORES_VAL"
VALIDATION_4RANKS_VAL = "4RANKING_VAL"
VALIDATION_2STAGE = "2STAGE_VAL"  # opitz - 1 criteria for 1 cq


def load_last_state_and_validate(arg):
    agents = [x for x in arg["round_answer_dict"].keys() if x.startswith("agent")]
    arg_cqs = []
    for agent in agents:
        last_cq_list = arg["round_answer_dict"][agent][-1]["critical_question_list"][
            "critical_questions"
        ]
        agent_cq = list(set([cq["critical_question"] for cq in last_cq_list]))
        arg_cqs.extend(agent_cq)

    arg_cqs = list(set(arg_cqs))
    original_arg = arg["input_arg"]

    return original_arg, arg_cqs


def revalidate_cqs(val_type, original_arg, arg_cqs, arg_id_):
    print(len(arg_cqs))
    if len(arg_cqs) <= 3:
        new_cqs = [{"cq": x, "id": i} for i, x in enumerate(arg_cqs)]

    elif val_type == VALIDATION_4SCORES_VAL:
        # init workflow
        validator_workflow = ValidatorAgentBuilder(
            model_thread_id=f"{best_experiment_setting}_{val_type}",
            llm_name=llm_full,
            llm_num=1,
            experiment_name=f"{best_experiment_setting}_{val_type}",
            temperature=0.7,
            cqs=arg_cqs,
        )
        # display(Image(validator_workflow.graph.get_graph(xray=1).draw_mermaid_png()))
        sorted_questions = validator_workflow._invoke_graph(
            {"input_arg": original_arg}, arg_id_
        ).model_dump()["critical_questions"]
        new_cqs = [
            {"cq": x["critical_question"], "id": x["id"]} for x in sorted_questions
        ]
    elif val_type == VALIDATION_2STAGE:
        # init workflow
        ranker_flow = TwoStepsCriteriaScorer(
            model_thread_id=f"{val_type}_{best_experiment_setting}",
            llm_name=llm_full,
            llm_num=1,
            experiment_name=f"{val_type}_{best_experiment_setting}",
            temperature=0.7,
        )

        sorted_questions = ranker_flow._invoke_graph(
            {"input_arg": original_arg, "cqs": arg_cqs}, arg_id_
        ).model_dump()["critical_questions"]
        new_cqs = [
            {"cq": x["critical_question"], "id": x["id"]} for x in sorted_questions
        ]
    elif val_type == VALIDATION_4RANKS_VAL:
        # init workflow
        ranker_flow = RankerAgentBuilder(
            model_thread_id=f"{val_type}_{best_experiment_setting}",
            llm_name=llm_full,
            llm_num=1,
            experiment_name=f"{val_type}_{best_experiment_setting}",
            temperature=0.7,
        )
        # try:
        # display(Image(ranker_flow.graph.get_graph(xray=1).draw_mermaid_png()))
        arg_cqs_str = "-" + "\n-".join(arg_cqs)
        sorted_questions = ranker_flow._invoke_graph(
            {"input_arg": original_arg, "cqs": arg_cqs_str}, arg_id_
        ).model_dump()["critical_questions"]
        new_cqs = [
            {"cq": x["critical_question"], "id": x["id"]} for x in sorted_questions
        ]
    return new_cqs


# SET VALS FOR EXPERIMENT
val_type = VALIDATION_4RANKS_VAL
OUTPUT_FILES = "output/elbaff_experiment"

for a_best in the_bests:
    best_experiment_setting = a_best
    llm_key = best_experiment_setting.split("_")[0]
    llm_full = llm_dic[f"{llm_key}_"]

    out_f = f"{OUTPUT_FILES}/output_{best_experiment_setting}.json"

    new_out_f = f"{OUTPUT_FILES}/output_{val_type}_{best_experiment_setting}.json"
    arg_path = f"{OUTPUT_FILES}/final_states/{best_experiment_setting}_arg{{}}.json"

    with open(out_f, "r") as file:
        new_out = {}
        json_ = json.load(file)
        print(json_.keys())
        for arg_id_ in json_.keys():

            new_out[arg_id_] = json_[arg_id_]
            new_out[arg_id_]["cqs"] = []
            # each arg
            with open(arg_path.format(arg_id_), "r") as file:
                arg = json.load(file)
                original_arg, cqs = load_last_state_and_validate(arg)
                not_done = True
                retries = 0
                while not_done and retries < 10:
                    try:
                        retries = retries + 1
    
                        new_out[arg_id_]["cqs"] = revalidate_cqs(
                            val_type, original_arg, cqs, arg_id_
                        )
                        not_done = False
                    except Exception as e:
                        print(e)
                        not_done = True

        # SAVE NEW
        print(f"saving to {new_out_f}")
        with open(new_out_f, "w") as nof:
            json.dump(new_out, nof, indent=4)  # pretty print the json

dict_keys(['CLINTON_199_2', 'CLINTON_1_2', 'CLINTON_21', 'CLINTON_223_1', 'CLINTON_223_2', 'CLINTON_225', 'CLINTON_231', 'CLINTON_235', 'CLINTON_244_2', 'CLINTON_25', 'CLINTON_37', 'CLINTON_39', 'CLINTON_55', 'CLINTON_57', 'CLINTON_66', 'CLINTON_6_1', 'CLINTON_6_2', 'CLINTON_85', 'CLINTON_87', 'CLINTON_92', 'CL_57', 'Doctor-Mom_205', 'Elmattador__92', 'FoodAllergyMom_199_1', 'Frequent-Flyer_157', 'Glblwrmingisfak__552', 'HOLT_126', 'HOLT_239', 'HOLT_94', 'Helen__66', '17th_knight__247', 'AFCHF_154', 'AK-traveler_186', 'AllergyDad_200', 'AngelComa__638', 'Antanagoge_104', 'Antanagoge_111', 'Antanagoge_227', 'Antanagoge_239', 'Bill_106', 'CF_29', 'CF_46', 'CLINTON_103', 'CLINTON_123_1', 'CLINTON_123_2', 'CLINTON_130_1', 'CLINTON_130_2', 'CLINTON_147', 'CLINTON_17', 'CLINTON_176_1', 'CLINTON_176_2', 'CLINTON_186', 'CLINTON_193', 'CLINTON_199_1', 'TRUMP_240_2', 'TRUMP_249', 'TRUMP_251', 'TRUMP_253', 'TRUMP_26', 'TRUMP_275', 'TRUMP_279', 'TRUMP_3_1', 'TRUMP_3_2', 'TRUMP_44', 'TRUMP_52', 'TR